In [1]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

class SlackClient:
    def __init__(self):
        self.client = WebClient(token=os.environ['SLACK_BOT_TOKEN'])
        self.channel_id = os.environ['SLACK_CHANNEL_ID']

    def post_message(self, message):
        try:
            response = self.client.chat_postMessage(channel=self.channel_id, text=message)
            assert response["message"]["text"] == message
        except SlackApiError as e:
            print(f"Error posting message: {e.response['error']}")


In [2]:
def test_slack():
    slack_client = SlackClient()
    slack_client.post_message('Hello, world!')


In [3]:
test_slack()


In [4]:
import openai

class OpenAIClient:
    def __init__(self):
        openai.api_key = os.environ['OPENAI_KEY']

    def get_response(self, prompt, max_tokens=150):
        try:
            response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, max_tokens=max_tokens)
            return response.choices[0].text.strip()
        except Exception as e:
            print(f"Error in generating response: {str(e)}")
            return None


In [5]:
def test_openai():
    openai_client = OpenAIClient()
    response = openai_client.get_response('Hello, world!')
    print(response)


In [6]:
test_openai()

This is my first GitHub post.


In [7]:
!pip install requests

In [8]:
import requests
import os

class HomeAssistantClient:
    def __init__(self):
        self.url = os.environ['HA_SERVER']
        self.headers = {
            "Authorization": f"Bearer {os.environ['HA_TOKEN']}",
            "Content-Type": "application/json"
        }

    def get_state(self, entity_id):
        try:
            response = requests.get(
                f"{self.url}/api/states/{entity_id}",
                headers=self.headers
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error in getting state: {str(e)}")
            return None

    def set_state(self, entity_id, state):
        try:
            response = requests.post(
                f"{self.url}/api/states/{entity_id}",
                headers=self.headers,
                json={"state": state},
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error in setting state: {str(e)}")
            
    def get_all_states(self):
        try:
            response = requests.get(
                f"{self.url}/api/states",
                headers=self.headers,
            )
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error in getting all states: {str(e)}")
            return None


In [9]:
def test_homeassistant():
    ha_client = HomeAssistantClient()
    state = ha_client.get_state('light.Entry')
    print(state)

test_homeassistant()


{'entity_id': 'light.entry', 'state': 'off', 'attributes': {'supported_color_modes': ['brightness'], 'is_hue_group': True, 'hue_scenes': ['Nightlight', 'Concentrate', 'Relax', 'Energize', 'Dimmed', 'Read', 'Bright'], 'hue_type': 'room', 'lights': ['Entryway'], 'dynamics': False, 'icon': 'mdi:lightbulb-group', 'friendly_name': 'Entry', 'supported_features': 40}, 'last_changed': '2023-05-22T13:37:04.444689+00:00', 'last_updated': '2023-05-22T13:37:04.444689+00:00', 'context': {'id': '01H11TV0DWEPY0R6RR9HFFWESV', 'parent_id': None, 'user_id': None}}


In [10]:
!pip install influxdb-client

  Using cached influxdb_client-1.36.1-py3-none-any.whl (721 kB)
  Using cached reactivex-4.0.4-py3-none-any.whl (217 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)


In [11]:
def list_HA_entities():
    ha_client = HomeAssistantClient()
    entities = ha_client.get_all_states()
    for entity in entities:
        print(entity['entity_id'])

list_HA_entities()

person.jason_arends
binary_sensor.above_horizon
update.home_assistant_supervisor_update
update.home_assistant_core_update
update.let_s_encrypt_update
update.file_editor_update
update.terminal_ssh_update
update.home_assistant_operating_system_update
sensor.gecko_sunrise
sensor.gecko_sunset
sensor.gecko_daylight
sensor.gecko_sun_phase
sensor.qbittorrent_status
sensor.qbittorrent_down_speed
sensor.moon_phase
sun.sun
zone.home
sensor.qbittorrent_up_speed
binary_sensor.pihole2
sensor.pihole2_ads_blocked_today
sensor.pihole2_ads_percentage_blocked_today
sensor.pihole2_seen_clients
sensor.pihole2_dns_queries_today
sensor.pihole2_domains_blocked
sensor.pihole2_dns_queries_cached
sensor.pihole2_dns_queries_forwarded
sensor.pihole2_dns_unique_clients
sensor.pihole2_dns_unique_domains
update.pihole2_core_update_available
update.pihole2_web_update_available
update.pihole2_ftl_update_available
binary_sensor.dietpi
sensor.dietpi_ads_blocked_today
sensor.dietpi_ads_percentage_blocked_today
sensor.die

In [12]:
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.flux_table import FluxTable
import pandas as pd

class InfluxDBConnection:
    def __init__(self):
        self.client = InfluxDBClient(
            url=os.environ['INFLUX_SERVER'],
            token=os.environ['INFLUX_TOKEN'],
            org=os.environ['INFLUX_ORG']
        )

    def query(self, query):
        try:
            tables = self.client.query_api().query(query, org=self.client.org)
            result = []
            for table in tables:
                for record in table.records:
                    result.append(record.values)
            return result
        except Exception as e:
            print(f"Error in query: {str(e)}")
            return None

    def query_df(self, query):
        try:
            df = self.client.query_api().query_data_frame(query, org=self.client.org)
            return df
        except Exception as e:
            print(f"Error in query: {str(e)}")
            return None

    def write(self, bucket, record):
        try:
            write_api = self.client.write_api(write_options=SYNCHRONOUS)
            write_api.write(bucket, self.client.org, record)
            write_api.__del__()
        except Exception as e:
            print(f"Error in write: {str(e)}")

    def get_health(self):
        return self.client.health()

    def close(self):
        self.client.__del__()



In [14]:
def test_influxdb_connection():
    # Instantiate the InfluxDBConnection
    influx_conn = InfluxDBConnection()

    # Test query()
    query = f'from(bucket: "{os.environ["INFLUX_HA_BUCKET"]}")|> range(start: -1h)|> filter(fn: (r) => r["_measurement"] == "temperature")'
    result = influx_conn.query(query)
    print(result)

    # Test query_df()
    result_df = influx_conn.query_df(query)
    print(result_df)

    # Test write()
    # Assuming you have a Point or a list of Points to write
    #points = [Point("measurement").tag("location", "west").field("temperature", 20.3)]
    #bucket = '<Your bucket here>'
    #influx_conn.write(bucket, points)

    # Test get_health()
    health = influx_conn.get_health()
    print(health)

    # Close the connection
    influx_conn.close()

test_influxdb_connection()


[]
Empty DataFrame
Columns: []
Index: []
{'checks': [],
 'commit': '407fa622e9',
 'message': 'ready for queries and writes',
 'name': 'influxdb',
 'status': 'pass',
 'version': 'v2.7.1'}


/opt/HomeControlHub/venv/lib/python3.9/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "home_assistant")|> range(start: -1h)|> filter(fn: (r) => r["_measurement"] == "temperature") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


In [15]:
query = f'''
import "influxdata/influxdb/schema"
schema.measurements(bucket: "{os.environ['INFLUX_HA_BUCKET']}")
'''
influx = InfluxDBConnection()
df = influx.query_df(query)
print(df)


      result  table                 _value
0    _result      0                      %
1    _result      0                     Hz
2    _result      0                  Index
3    _result      0                  KiB/s
4    _result      0                Minutes
..       ...    ...                    ...
226  _result      0  weather.kjln_daynight
227  _result      0     weather.my_ecobee3
228  _result      0                      °
229  _result      0                     °C
230  _result      0                     °F

[231 rows x 3 columns]


/opt/HomeControlHub/venv/lib/python3.9/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
import "influxdata/influxdb/schema"
schema.measurements(bucket: "home_assistant")
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


In [17]:
query = f'''
from(bucket: "{os.environ['INFLUX_HA_BUCKET']}")
  |> range(start: -1d)
  |> filter(fn: (r) => r["_measurement"] == "weather.my_ecobee3")
'''
df = influx.query_df(query)
print(df)


/opt/HomeControlHub/venv/lib/python3.9/site-packages/influxdb_client/client/warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
from(bucket: "home_assistant")
  |> range(start: -1d)
  |> filter(fn: (r) => r["_measurement"] == "weather.my_ecobee3")
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


[       result  table                           _start  \
0     _result      0 2023-05-21 16:38:03.836830+00:00   
1     _result      0 2023-05-21 16:38:03.836830+00:00   
2     _result      0 2023-05-21 16:38:03.836830+00:00   
3     _result      0 2023-05-21 16:38:03.836830+00:00   
4     _result      0 2023-05-21 16:38:03.836830+00:00   
...       ...    ...                              ...   
2875  _result      0 2023-05-21 16:38:03.836830+00:00   
2876  _result      0 2023-05-21 16:38:03.836830+00:00   
2877  _result      0 2023-05-21 16:38:03.836830+00:00   
2878  _result      0 2023-05-21 16:38:03.836830+00:00   
2879  _result      0 2023-05-21 16:38:03.836830+00:00   

                                _stop                            _time  \
0    2023-05-22 16:38:03.836830+00:00 2023-05-21 16:38:19.432173+00:00   
1    2023-05-22 16:38:03.836830+00:00 2023-05-21 16:38:48.825843+00:00   
2    2023-05-22 16:38:03.836830+00:00 2023-05-21 16:39:18.826196+00:00   
3    2023-05-22 16